<a href="https://colab.research.google.com/github/actualIslam/T5-tasks/blob/main/Copy_of_Task1_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continuous Bag of Words (CBOW) Model

In this exam, we will create a CBOW model using a sample Arabic traffic corpus. The corpus consists of sentences describing various traffic scenarios. The goal of the CBOW model is to predict a target word based on its surrounding context words.

# Importing Required Libraries

In this step, we import the necessary libraries to build and train the Continuous Bag of Words (CBOW) model.

- **TensorFlow and Keras**: Used to build the neural network model, including the layers like `Embedding`, `Dense`, and `Lambda`.
- **Tokenizer**: A utility from Keras for tokenizing and processing text data.
- **NumPy**: Used for handling numerical operations, particularly for processing arrays and data manipulation.

These libraries will provide the essential tools for text preprocessing and model development in the upcoming steps.

Add more if needed!


In [2]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.9 MB/s eta 0:00:00


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Embedding, Lambda
from keras.models import Sequential
from pyarabic.araby import strip_tashkeel

# Preparing the Corpus

In this step, we initialize the corpus that will be used for training the Continuous Bag of Words (CBOW) model. The corpus consists of Arabic sentences, each of which describes different traffic scenarios.

- **Corpus**: A collection of traffic-related sentences in Arabic.

This step sets up the text data that we will use in the upcoming stages of tokenization and model training.

In [4]:
corpus = [
    "الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول",
    "كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن",
    "أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق",
    "السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة",
    "هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير",
    "ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل",
    "الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير",
    "الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم",
    "حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك",
    "كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذي استمر طوال اليوم",
    "الطريق إلى المطار مزدحم اليوم بسبب الحوادث المتكررة والتأخيرات الكبيرة في حركة المرور على الطريق السريع",
    "الشارع مزدحم بالسيارات والحافلات الكبيرة مما يجعل التنقل بطيئًا جدًا ويزيد من وقت الوصول إلى العمل",
    "ازدحام السيارات في المدينة أصبح مشكلة كبيرة خاصة خلال ساعات الذروة حيث يصعب التحرك بسرعة",
    "تفاقم الازدحام في الطرق الجانبية بسبب إغلاق الطريق الرئيسي المؤدي إلى وسط المدينة لصيانة الجسر",
    "ازدحام مروري خانق يواجه السكان يوميًا خلال تنقلهم من وإلى العمل على الطرق السريعة المؤدية إلى المدينة",
    "التأخيرات المرورية اليوم ناجمة عن سوء الأحوال الجوية والضباب الذي يعيق الرؤية ويبطئ حركة السيارات",
    "حوادث السير المتكررة على الطريق الزراعي تؤدي إلى ازدحام مروري شديد وتأخير كبير في وصول السيارات",
    "كانت حركة السير اليوم غير منتظمة بسبب تنظيم حدث رياضي كبير أدى إلى إغلاق بعض الشوارع الرئيسية",
    "الأعمال الإنشائية في الشارع الرئيسي تسببت في اختناق مروري كامل وتباطؤ في حركة السيارات خلال النهار",
    "تراكم السيارات عند تقاطع الطرق الرئيسية أدى إلى ازدحام شديد وزيادة كبيرة في مدة الانتظار للوصول إلى الجهة المطلوبة"
]

# Defining Vocabulary and Model Parameters

In this step, we define key parameters that will be used to configure the CBOW model.

- **Vocabulary size**: We calculate the size of the vocabulary based on the number of unique words in the corpus. The `vocab_size` represents the total number of unique tokens (words) in the dataset plus one for padding.
  
- **Embedding size**: The `embedding_size` defines the dimensionality of the word embeddings. In this case, we set the embedding size to 10, meaning each word will be represented as a 10-dimensional vector in the embedding layer.

- **Window size**: The `window_size` defines how many words to the left and right of the target word are considered as context. Here, a window size of 2 means that two words before and two words after the target word will be used as context.

These parameters will play an essential role in shaping the CBOW model architecture.


In [5]:
def unique_words(data):
  text = ""
  for sen in data:
    text += sen + ' '

  return set(text.split())

# This function is as we learned in past week with some change because we deal here with a list not a pandas DataFrame,
# for that I don't have a link to paste it here

corpus_unique = unique_words(corpus)
corpus_unique

{'أحب',
 'أدى',
 'أصبح',
 'أعمال',
 'أنا',
 'إغلاق',
 'إلى',
 'اختناق',
 'ازدحام',
 'ازدحاما',
 'استمر',
 'الأحوال',
 'الأعمال',
 'الإنشائية',
 'الازدحام',
 'الانتظار',
 'الباكر',
 'البناء',
 'التأخيرات',
 'التجارية',
 'التحرك',
 'التنقل',
 'التي',
 'الجانبية',
 'الجسر',
 'الجميع',
 'الجهة',
 'الجوية',
 'الحادث',
 'الحافلات',
 'الحوادث',
 'الخانق',
 'الخضروات',
 'الذروة',
 'الذهاب',
 'الذي',
 'الرؤية',
 'الرئيسي',
 'الرئيسية',
 'الزراعي',
 'السريع',
 'السريعة',
 'السكان',
 'السوق',
 'السيارات',
 'السير',
 'الشاحنات',
 'الشارع',
 'الشديد',
 'الشوارع',
 'الصباح',
 'الصعب',
 'الطازجة',
 'الطرق',
 'الطريق',
 'الظهيرة',
 'العمل',
 'الفعاليات',
 'القيادة',
 'الكبيرة',
 'المؤدي',
 'المؤدية',
 'المتكررة',
 'المدينة',
 'المرور',
 'المروري',
 'المرورية',
 'المساء',
 'المطار',
 'المطلوبة',
 'المنطقة',
 'الناس',
 'النهار',
 'الوصول',
 'اليوم',
 'بالسيارات',
 'بالعودة',
 'بالكامل',
 'بسبب',
 'بسرعة',
 'بطيئة',
 'بطيئًا',
 'بعض',
 'تأخير',
 'تؤدي',
 'تبطئ',
 'تتحسن',
 'تراكم',
 'تسببت',
 'تعطل',
 'ت

In [6]:
len(corpus_unique)

178

In [7]:
VOCAB_SIZE = len(corpus_unique) + 1

In [8]:
EMBEDDING_SIZE = 10

In [9]:
WINDOW_SIZE = 2

# Preparing Context-Target Pairs for CBOW

In this step, we generate the context-target pairs from the tokenized sequences to train the CBOW model.

- **Context words**: For each word in a sequence, the surrounding words (within the window size) are considered as context. The context consists of the words immediately before and after the target word.
  
- **Target word**: The word in the middle of the context window is treated as the target word that the model will learn to predict.

We iterate through each sequence, collecting the context words and corresponding target words:
- For each word in a sequence, we gather the surrounding words based on the defined window size.
- The middle word is the target, and the surrounding words form the context.

Finally:
- **`X`**: An array of context words.
- **`y`**: The target words are one-hot encoded, which means they are converted into a categorical format where each word is represented as a vector of length equal to the vocabulary size.

These context-target pairs will be used to train the CBOW model to predict a target word based on its context.


In [10]:
# firstly there is some تنوين needs to be stripped so I'm using pyarabic library to do so
stripped_corpus = [strip_tashkeel(sen) for sen in corpus] # --> https://pyarabic.readthedocs.io/ar/latest/features.html
stripped_corpus

['الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحا ويؤدي إلى تأخير كبير في الوصول',
 'كان الطريق مزدحما للغاية والسيارات متوقفة تقريبا نتيجة الازدحام الشديد والحركة بطيئة جدا ولا تتحسن',
 'أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق',
 'السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة',
 'هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير',
 'ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل',
 'الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير',
 'الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم',
 'حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك',
 'كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخان

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
from nltk.tokenize import word_tokenize

In [13]:
# using word_tokenize instead of .split()
tokenized = [word_tokenize(sen) for sen in stripped_corpus] # --> https://www.nltk.org/api/nltk.tokenize.html
tokenized

[['الطريق',
  'مزدحم',
  'اليوم',
  'بسبب',
  'الحادث',
  'المروري',
  'الذي',
  'حدث',
  'صباحا',
  'ويؤدي',
  'إلى',
  'تأخير',
  'كبير',
  'في',
  'الوصول'],
 ['كان',
  'الطريق',
  'مزدحما',
  'للغاية',
  'والسيارات',
  'متوقفة',
  'تقريبا',
  'نتيجة',
  'الازدحام',
  'الشديد',
  'والحركة',
  'بطيئة',
  'جدا',
  'ولا',
  'تتحسن'],
 ['أنا',
  'أحب',
  'الذهاب',
  'إلى',
  'السوق',
  'في',
  'الصباح',
  'الباكر',
  'لتفادي',
  'الازدحام',
  'وشراء',
  'الخضروات',
  'الطازجة',
  'دون',
  'الانتظار',
  'في',
  'الطريق'],
 ['السيارات',
  'بطيئة',
  'بسبب',
  'الازدحام',
  'المروري',
  'في',
  'الشارع',
  'الرئيسي',
  'والتأخير',
  'في',
  'حركة',
  'المرور',
  'خلال',
  'ساعات',
  'الذروة'],
 ['هناك',
  'ازدحام',
  'مروري',
  'في',
  'الشارع',
  'بسبب',
  'أعمال',
  'البناء',
  'والحفريات',
  'التي',
  'تعطل',
  'حركة',
  'السيارات',
  'وتتسبب',
  'في',
  'تأخير',
  'كبير'],
 ['ازدحام',
  'السيارات',
  'يزداد',
  'في',
  'المساء',
  'عندما',
  'يبدأ',
  'الجميع',
  'بالعودة',
  'إلى',
  

In [25]:
# This is the best I could do for the context. Because of that first two words in each sentence would be without context sadly

X = []
y = []

for sen in tokenized:
  for word in sen:
    word_index = sen.index(word)
    context = sen[word_index-WINDOW_SIZE:word_index+WINDOW_SIZE+1]

    if word not in context:
      continue
    else:
      context.remove(word)

    X.append(context)
    y.append(word)

X, y

([['الطريق', 'مزدحم', 'بسبب', 'الحادث'],
  ['مزدحم', 'اليوم', 'الحادث', 'المروري'],
  ['اليوم', 'بسبب', 'المروري', 'الذي'],
  ['بسبب', 'الحادث', 'الذي', 'حدث'],
  ['الحادث', 'المروري', 'حدث', 'صباحا'],
  ['المروري', 'الذي', 'صباحا', 'ويؤدي'],
  ['الذي', 'حدث', 'ويؤدي', 'إلى'],
  ['حدث', 'صباحا', 'إلى', 'تأخير'],
  ['صباحا', 'ويؤدي', 'تأخير', 'كبير'],
  ['ويؤدي', 'إلى', 'كبير', 'في'],
  ['إلى', 'تأخير', 'في', 'الوصول'],
  ['تأخير', 'كبير', 'الوصول'],
  ['كبير', 'في'],
  ['كان', 'الطريق', 'للغاية', 'والسيارات'],
  ['الطريق', 'مزدحما', 'والسيارات', 'متوقفة'],
  ['مزدحما', 'للغاية', 'متوقفة', 'تقريبا'],
  ['للغاية', 'والسيارات', 'تقريبا', 'نتيجة'],
  ['والسيارات', 'متوقفة', 'نتيجة', 'الازدحام'],
  ['متوقفة', 'تقريبا', 'الازدحام', 'الشديد'],
  ['تقريبا', 'نتيجة', 'الشديد', 'والحركة'],
  ['نتيجة', 'الازدحام', 'والحركة', 'بطيئة'],
  ['الازدحام', 'الشديد', 'بطيئة', 'جدا'],
  ['الشديد', 'والحركة', 'جدا', 'ولا'],
  ['والحركة', 'بطيئة', 'ولا', 'تتحسن'],
  ['بطيئة', 'جدا', 'تتحسن'],
  ['جدا', 'ولا

In [30]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

y_encoded = one_hot.fit_transform([y])

# Building and Training the CBOW Model

In this step, we build and train the Continuous Bag of Words (CBOW) model using the context-target pairs created earlier.

1. **Model architecture**:
   - **Embedding layer**: This layer transforms the input context words into dense vector representations (embeddings) of size defined by `embedding_size`. The `input_dim` is set to the vocabulary size, and the `input_length` is twice the window size (since context consists of words from both sides of the target).
   
   - **Lambda layer**: This layer computes the mean of the context word embeddings. It averages the embeddings of the context words to generate a single representation that will be used to predict the target word.
   
   - **Dense layer**: This fully connected layer outputs a probability distribution over the vocabulary, using the softmax activation function. It predicts the most likely target word based on the context word embeddings.

2. **Compilation**:
   The model is compiled using the Adam optimizer and categorical cross-entropy as the loss function, which is suitable for multi-class classification tasks. Accuracy is used as a metric to evaluate the model's performance during training.

3. **Training the model**:
   The model is trained on the context-target pairs for 500 epochs. During each epoch, the model learns to predict the target word based on the context, refining its weights to improve accuracy.

4. **Saving the model weights**:
   After training, the model weights are saved to a file (`cbow_model.weights.h5`) for future use. This allows us to load the trained model later without retraining.

By the end of this step, the CBOW model will have learned to predict target words based on their surrounding context from the given corpus.

In [35]:
# Arch
model = Sequential([Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=20,
    input_length=WINDOW_SIZE ** 2,
    embeddings_initializer="uniform",),
                    Dense(72, activation='softmax')])

# compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# fit
model.fit()

# Predicting a Word Using the Trained CBOW Model

In this step, we define a function to predict a word based on a given context using the trained CBOW model.

1. **Function: `predict_word`**:
   - **Input**: The function takes a list of context words as input. The number of context words should match the expected size (2 times the window size).
   - **Context sequence conversion**: The input context words are tokenized into a sequence of integers using the same tokenizer that was used during training.
   - **Input validation**: The function checks whether the length of the context sequence matches the expected size (2 times the window size). If not, it prints an error message.
   - **Prediction**: The tokenized context is fed into the trained CBOW model, which predicts the probability distribution over the vocabulary.
   - **Retrieve predicted word**: The predicted word is the one with the highest probability. The function retrieves the word corresponding to the predicted index from the tokenizer's word index.

2. **Example**:
   - We provide a sample context: `['الحادث', 'بسبب', 'مزدحم', 'الطريق']`.
   - The function predicts the word that fits best in this context, based on the model's learned weights.
   - The predicted word is printed along with the input context.

This function allows us to test the CBOW model by predicting words based on their surrounding context from the corpus.



##### Even if my grade was zero I will be satisfied with it as I'm not satisfied with my solutions for both notebooks but I can assure you that I learned a lot from you, I just didn't study and this is not an execuse for sure.

##### ولأن من لا يشكر الناس لا يشكر الله شكرًا لك أستاذ علي كنت خير معلم